In [310]:
import sklearn
from sklearn.utils import shuffle
import pandas as pd
import csv
import time

In [311]:
trainingDataSet = []


df = pd.read_csv("tweetDataFile.csv", encoding = 'ISO-8859-1')
df.to_csv("tweetFile.csv", header = ['id','text','label','topics'])

In [312]:
df = pd.read_csv("tweetFile.csv", encoding = 'ISO-8859-1')
df = shuffle(df)
df.reset_index(inplace = True)
df.head()
df.drop(['index'], inplace = True, axis = 1)


In [313]:
df.head()
df.groupby('label').count()

,Unnamed: 0,id,text,topics
label,,,,
irrelevant,558,558,558,558
negative,389,389,389,389
neutral,1414,1414,1414,1414
positive,359,359,359,359


In [314]:
df.shape

(2722, 5)

In [315]:
X_train = df['text']
y_train = df['label']

X_train.head()

0    RT @centroandroid: #Google presenta #Android B...
1    Android ice-cream sandwich's "Face Unlock" fai...
2    @eboyee S tim da http://t.co/hX21e1JK daje Ser...
3    @Apple would do better to inform customers pro...
4    RT @ralphbin: #Microsoft Research shows #Holod...
Name: text, dtype: object

In [316]:
y_train.loc[1129]

'neutral'

In [317]:
import re
import string

X_train = X_train.apply(lambda x : x.lower())
X_train = X_train.apply(lambda x : re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', x))
X_train = X_train.apply(lambda x : re.sub('@[^\s]+', '', x))
X_train = X_train.apply(lambda x : x.replace('[^\w\s]',' '))
X_train = X_train.apply(lambda x :re.sub(r'#([^\s]+)', r'\1', x))
X_train = X_train.apply(lambda x : re.sub('[^A-Za-z0-9 ]', '', x))

# y_train = y_train.apply(lambda x : x.replace(' ','', x))


In [318]:
import sklearn.feature_extraction.text as ext
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer

#initializing the stemmer
stemmer = PorterStemmer()
def stem_tokens(tokens):
    stemmed = []
    for item in tokens:
        #append the stemmed item to list and return it
        stemmed.append(stemmer.stem(item))
    return stemmed
def tokenize(text):
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens)
    return stems

In [319]:
vect = ext.CountVectorizer(tokenizer = tokenize, stop_words = 'english', min_df = 5).fit(X_train)


C:\Users\Daniel Ogbuti\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.

In [320]:
text_train = vect.transform(X_train)
text_train.shape

(2722, 902)

In [321]:
from sklearn import preprocessing

In [322]:
y = {'positive':0,'negative':1,'neutral':2,'irrelevant':2,}
y_train.fillna('irrelevant',inplace = True)
y_train = y_train.map(y)
y_train[y_train.isnull()]

Series([], Name: label, dtype: int64)

In [323]:
y_train.head()

0    2
1    1
2    2
3    1
4    0
Name: label, dtype: int64

In [324]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(text_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [325]:
print(classifier.predict(text_train))
print(y_train.values)

[2 2 2 ... 2 2 2]
[2 1 2 ... 2 2 2]


In [326]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pred = classifier.predict(text_train)
print(classification_report(y_train, pred))
print('Accuracy: ', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       0.72      0.68      0.70       359
           1       0.60      0.79      0.68       389
           2       0.91      0.86      0.89      1974

   micro avg       0.83      0.83      0.83      2722
   macro avg       0.74      0.78      0.76      2722
weighted avg       0.84      0.83      0.83      2722

Accuracy:  0.8277002204261572
